# Model v2.1 - Tensorflow Version

In [1]:
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd
import numpy as np

In [5]:
version = 2.1
version_str = "v" + str(version)

### Load dataset

In [243]:
LABEL = "price_doc"

In [443]:
def prepare_dataset(df, test=False, trunc_fields=False):
    global LABEL
    t_fields = ['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor','material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', LABEL]
    
    if not test:
        if trunc_fields:
            df = df[t_fields + LABEL]
        else:
            df = df
    else:
        if trunc_fields:
            df = df[t_fields]
        else:
            df = df
            
    fix_cols = [c for c in df.columns.tolist() if '+' in c]
    
    df['timestamp_converted'] = pd.to_datetime(df['timestamp'])
    df['timestamp_day'] = df.apply(lambda row: row['timestamp_converted'].day, axis=1)
    df['timestamp_month'] = df.apply(lambda row: row['timestamp_converted'].month, axis=1)
    df['timestamp_year'] = df.apply(lambda row: row['timestamp_converted'].year, axis=1)
    df['state_refractored'] = df.apply( lambda row: 5 if row['state'] == 33 else row['state'] , axis = 1)
    
    for c in fix_cols:
        df[c.replace("+","_")] = df[c]
        del df[c]
    
    df.fillna(0, inplace=True)
    
    df['state_refractored'] = df['state_refractored'].astype(int)    
    df['material'] = df['material'].astype(int)    
    return df

In [444]:
final_train_df = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/final_training_dataset.csv")

In [445]:
final_train_df = prepare_dataset(final_train_df)

In [446]:
del final_train_df['Unnamed: 0']

In [447]:
final_train_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,rent_price_4_room_bus
0,1,2011-08-20,43,27.0,4.0,0.0,0,0.0,0.0,0.0,...,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,136.11
1,2,2011-08-23,34,19.0,3.0,0.0,0,0.0,0.0,0.0,...,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,136.11
2,3,2011-08-27,43,29.0,2.0,0.0,0,0.0,0.0,0.0,...,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,136.11
3,4,2011-09-01,89,50.0,9.0,0.0,0,0.0,0.0,0.0,...,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,155.22
4,5,2011-09-05,77,77.0,4.0,0.0,0,0.0,0.0,0.0,...,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0,155.22


In [448]:
[c for c in final_train_df.columns.tolist() if '+' in c]

[]

## Data prep

In [328]:
def get_bin_boundaries(df, col):
    step = int(df[col].std())  
    if step == 0:
        return [float(df[col].min()), float(df[col].max())]
    else:
        return [float(g) for g in list(range( int(df[col].min()), int(df[col].max()), step))]

In [329]:
def bucketize_column(df, col, bins=[]):
    if not bins:
        bins = get_bin_boundaries(df, col)
        if bins.size == 0:
            bins = [df[col].min(), df[col].max()]
    return np.digitize(df[col].values, bins, right=True)

### Notes
- ID_* columns should be embedded!
- _1line are no/yes columns, so they should be labeled...
- "ecology" has values of ['good', 'excellent', 'poor', 'satisfactory', 'no data'],... labelled/one-hot encoded/or embedded?
- church_count_500 and other _count values integer values - bucketize them?
- timestamp_day, timestamp_month, timestamp_year

In [330]:
import math
def determine_dimensions(num_unique, r = 0, k = 1):
    if r == 0:
        return int(math.log(num_unique, 2))
    else:
        return k * int(num_unique ** 1./4.)

In [331]:
def dedup(listy):
    return list(set(listy))

In [449]:
all_feature_columns = [
 'full_sq',
 'life_sq',
 'floor',
 'max_floor',
 'material',
 'build_year',
 'num_room',
 'kitch_sq',
 'state',
 'product_type',
 'sub_area',
 'area_m',
 'raion_popul',
 'green_zone_part',
 'indust_part',
 'children_preschool',
 'preschool_quota',
 'preschool_education_centers_raion',
 'children_school',
 'school_quota',
 'school_education_centers_raion',
 'school_education_centers_top_20_raion',
 'hospital_beds_raion',
 'healthcare_centers_raion',
 'university_top_20_raion',
 'sport_objects_raion',
 'additional_education_raion',
 'culture_objects_top_25',
 'culture_objects_top_25_raion',
 'shopping_centers_raion',
 'office_raion',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'full_all',
 'male_f',
 'female_f',
 'young_all',
 'young_male',
 'young_female',
 'work_all',
 'work_male',
 'work_female',
 'ekder_all',
 'ekder_male',
 'ekder_female',
 '0_6_all',
 '0_6_male',
 '0_6_female',
 '7_14_all',
 '7_14_male',
 '7_14_female',
 '0_17_all',
 '0_17_male',
 '0_17_female',
 '16_29_all',
 '16_29_male',
 '16_29_female',
 '0_13_all',
 '0_13_male',
 '0_13_female',
 'raion_build_count_with_material_info',
 'build_count_block',
 'build_count_wood',
 'build_count_frame',
 'build_count_brick',
 'build_count_monolith',
 'build_count_panel',
 'build_count_foam',
 'build_count_slag',
 'build_count_mix',
 'raion_build_count_with_builddate_info',
 'build_count_before_1920',
 'build_count_1921-1945',
 'build_count_1946-1970',
 'build_count_1971-1995',
 'build_count_after_1995',
 'ID_metro',
 'metro_min_avto',
 'metro_km_avto',
 'metro_min_walk',
 'metro_km_walk',
 'kindergarten_km',
 'school_km',
 'park_km',
 'green_zone_km',
 'industrial_km',
 'water_treatment_km',
 'cemetery_km',
 'incineration_km',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'ID_railroad_station_walk',
 'railroad_station_avto_km',
 'railroad_station_avto_min',
 'ID_railroad_station_avto',
 'public_transport_station_km',
 'public_transport_station_min_walk',
 'water_km',
 'water_1line',
 'mkad_km',
 'ttk_km',
 'sadovoe_km',
 'bulvar_ring_km',
 'kremlin_km',
 'big_road1_km',
 'ID_big_road1',
 'big_road1_1line',
 'big_road2_km',
 'ID_big_road2',
 'railroad_km',
 'railroad_1line',
 'zd_vokzaly_avto_km',
 'ID_railroad_terminal',
 'bus_terminal_avto_km',
 'ID_bus_terminal',
 'oil_chemistry_km',
 'nuclear_reactor_km',
 'radiation_km',
 'power_transmission_line_km',
 'thermal_power_plant_km',
 'ts_km',
 'big_market_km',
 'market_shop_km',
 'fitness_km',
 'swim_pool_km',
 'ice_rink_km',
 'stadium_km',
 'basketball_km',
 'hospice_morgue_km',
 'detention_facility_km',
 'public_healthcare_km',
 'university_km',
 'workplaces_km',
 'shopping_centers_km',
 'office_km',
 'additional_education_km',
 'preschool_km',
 'big_church_km',
 'church_synagogue_km',
 'mosque_km',
 'theater_km',
 'museum_km',
 'exhibition_km',
 'catering_km',
 'ecology',
 'green_part_500',
 'prom_part_500',
 'office_count_500',
 'office_sqm_500',
 'trc_count_500',
 'trc_sqm_500',
 'cafe_count_500',
 'cafe_sum_500_min_price_avg',
 'cafe_sum_500_max_price_avg',
 'cafe_avg_price_500',
 'cafe_count_500_na_price',
 'cafe_count_500_price_500',
 'cafe_count_500_price_1000',
 'cafe_count_500_price_1500',
 'cafe_count_500_price_2500',
 'cafe_count_500_price_4000',
 'cafe_count_500_price_high',
 'big_church_count_500',
 'church_count_500',
 'mosque_count_500',
 'leisure_count_500',
 'sport_count_500',
 'market_count_500',
 'green_part_1000',
 'prom_part_1000',
 'office_count_1000',
 'office_sqm_1000',
 'trc_count_1000',
 'trc_sqm_1000',
 'cafe_count_1000',
 'cafe_sum_1000_min_price_avg',
 'cafe_sum_1000_max_price_avg',
 'cafe_avg_price_1000',
 'cafe_count_1000_na_price',
 'cafe_count_1000_price_500',
 'cafe_count_1000_price_1000',
 'cafe_count_1000_price_1500',
 'cafe_count_1000_price_2500',
 'cafe_count_1000_price_4000',
 'cafe_count_1000_price_high',
 'big_church_count_1000',
 'church_count_1000',
 'mosque_count_1000',
 'leisure_count_1000',
 'sport_count_1000',
 'market_count_1000',
 'green_part_1500',
 'prom_part_1500',
 'office_count_1500',
 'office_sqm_1500',
 'trc_count_1500',
 'trc_sqm_1500',
 'cafe_count_1500',
 'cafe_sum_1500_min_price_avg',
 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500',
 'cafe_count_1500_na_price',
 'cafe_count_1500_price_500',
 'cafe_count_1500_price_1000',
 'cafe_count_1500_price_1500',
 'cafe_count_1500_price_2500',
 'cafe_count_1500_price_4000',
 'cafe_count_1500_price_high',
 'big_church_count_1500',
 'church_count_1500',
 'mosque_count_1500',
 'leisure_count_1500',
 'sport_count_1500',
 'market_count_1500',
 'green_part_2000',
 'prom_part_2000',
 'office_count_2000',
 'office_sqm_2000',
 'trc_count_2000',
 'trc_sqm_2000',
 'cafe_count_2000',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_count_2000_na_price',
 'cafe_count_2000_price_500',
 'cafe_count_2000_price_1000',
 'cafe_count_2000_price_1500',
 'cafe_count_2000_price_2500',
 'cafe_count_2000_price_4000',
 'cafe_count_2000_price_high',
 'big_church_count_2000',
 'church_count_2000',
 'mosque_count_2000',
 'leisure_count_2000',
 'sport_count_2000',
 'market_count_2000',
 'green_part_3000',
 'prom_part_3000',
 'office_count_3000',
 'office_sqm_3000',
 'trc_count_3000',
 'trc_sqm_3000',
 'cafe_count_3000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'cafe_count_3000_na_price',
 'cafe_count_3000_price_500',
 'cafe_count_3000_price_1000',
 'cafe_count_3000_price_1500',
 'cafe_count_3000_price_2500',
 'cafe_count_3000_price_4000',
 'cafe_count_3000_price_high',
 'big_church_count_3000',
 'church_count_3000',
 'mosque_count_3000',
 'leisure_count_3000',
 'sport_count_3000',
 'market_count_3000',
 'green_part_5000',
 'prom_part_5000',
 'office_count_5000',
 'office_sqm_5000',
 'trc_count_5000',
 'trc_sqm_5000',
 'cafe_count_5000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg',
 'cafe_avg_price_5000',
 'cafe_count_5000_na_price',
 'cafe_count_5000_price_500',
 'cafe_count_5000_price_1000',
 'cafe_count_5000_price_1500',
 'cafe_count_5000_price_2500',
 'cafe_count_5000_price_4000',
 'cafe_count_5000_price_high',
 'big_church_count_5000',
 'church_count_5000',
 'mosque_count_5000',
 'leisure_count_5000',
 'sport_count_5000',
 'market_count_5000',
 'timestamp_day',
 'timestamp_month',
 'timestamp_year',
 'state_refractored',
 'oil_urals',
 'gdp_quart',
 'gdp_quart_growth',
 'cpi',
 'ppi',
 'gdp_deflator',
 'balance_trade',
 'balance_trade_growth',
 'usdrub',
 'eurrub',
 'brent',
 'net_capital_export',
 'gdp_annual',
 'gdp_annual_growth',
 'average_provision_of_build_contract',
 'average_provision_of_build_contract_moscow',
 'rts',
 'micex',
 'micex_rgbi_tr',
 'micex_cbi_tr',
 'deposits_value',
 'deposits_growth',
 'deposits_rate',
 'mortgage_value',
 'mortgage_growth',
 'mortgage_rate',
 'grp',
 'grp_growth',
 'income_per_cap',
 'real_dispos_income_per_cap_growth',
 'salary',
 'salary_growth',
 'fixed_basket',
 'retail_trade_turnover',
 'retail_trade_turnover_per_cap',
 'retail_trade_turnover_growth',
 'labor_force',
 'unemployment',
 'employment',
 'invest_fixed_capital_per_cap',
 'invest_fixed_assets',
 'profitable_enterpr_share',
 'unprofitable_enterpr_share',
 'share_own_revenues',
 'overdue_wages_per_cap',
 'fin_res_per_cap',
 'marriages_per_1000_cap',
 'divorce_rate',
 'construction_value',
 'invest_fixed_assets_phys',
 'pop_natural_increase',
 'pop_migration',
 'pop_total_inc',
 'childbirth',
 'mortality',
 'housing_fund_sqm',
 'lodging_sqm_per_cap',
 'water_pipes_share',
 'baths_share',
 'sewerage_share',
 'gas_share',
 'hot_water_share',
 'electric_stove_share',
 'heating_share',
 'old_house_share',
 'average_life_exp',
 'infant_mortarity_per_1000_cap',
 'perinatal_mort_per_1000_cap',
 'incidence_population',
 'rent_price_4_room_bus',
 'rent_price_3room_bus',
 'rent_price_2room_bus',
 'rent_price_1room_bus',
 'rent_price_3room_eco',
 'rent_price_2room_eco',
 'rent_price_1room_eco',
 'load_of_teachers_preschool_per_teacher',
 'child_on_acc_pre_school',
 'load_of_teachers_school_per_teacher',
 'students_state_oneshift',
 'modern_education_share',
 'old_education_build_share',
 'provision_doctors',
 'provision_nurse',
 'load_on_doctors',
 'power_clinics',
 'hospital_beds_available_per_cap',
 'hospital_bed_occupancy_per_year',
 'provision_retail_space_sqm',
 'provision_retail_space_modern_sqm',
 'turnover_catering_per_cap',
 'theaters_viewers_per_1000_cap',
 'seats_theather_rfmin_per_100000_cap',
 'museum_visitis_per_100_cap',
 'bandwidth_sports',
 'population_reg_sports_share',
 'students_reg_sports_share',
 'apartment_build',
 'apartment_fund_sqm']

"""
Notes
ID_* columns should be embedded!
_1line are no/yes columns, so they should be labeled...
"ecology" has values of ['good', 'excellent', 'poor', 'satisfactory', 'no data'],... labelled/one-hot encoded/or embedded?
church_count_500 and other _count values integer values - bucketize them?
timestamp_day, timestamp_month, timestamp_year
"""

all_feature_columns = dedup(all_feature_columns)

_id_tag = "ID_"
_1line_tag = "_1line"
_raion_tag = "_raion"
_count_tag = "_count"
_km_tag = "_km"
_sq_tag = "_sq"

def get_subcolumns_by_tag(tag):
    global all_feature_columns
    fts = []
    for f in all_feature_columns:
        if tag in f:
            fts.append(f)
    return fts

def get_raion_columns(return_yesno=True, return_cont=False):
    global _raion_tag
    yesno_cols = []
    continuous_cols = []
    for c in get_subcolumns_by_tag(_raion_tag):
        if any(w in final_train_df[c].unique().tolist() for w in ['yes', 'no']):
            yesno_cols.append(c)
        else:
            continuous_cols.append(c)
    if return_yesno:
        return yesno_cols
    elif return_cont:
        return continuous_cols
    else:
        return yesno_cols, continuous_cols
    

integerized_embedding_columns = get_subcolumns_by_tag(_id_tag)
unlabeled_embedding_columns = ["ecology", "sub_area"]

embedding_columns = integerized_embedding_columns + unlabeled_embedding_columns


# Note: This will not modify the existing feature column(s).
# Rather, it will create a new supplemental feature column with bucketization
# Also, bucketized columns will be one-hot encoded, after being label encoded...
bucketize_columns = [
    # Specific columns to bucketize
] + get_subcolumns_by_tag(_count_tag) + get_subcolumns_by_tag(_km_tag) + get_subcolumns_by_tag(_sq_tag)

label_columns = [
    # Specific label columns to include...    
] + get_raion_columns(return_yesno=True) + get_subcolumns_by_tag(_1line_tag) + unlabeled_embedding_columns

one_hot_columns_to_be_labeled = [
    'material',
    'state_refractored',
    'product_type',
    'culture_objects_top_25',
]

integerized_label_columns = [
    'material',
    'state_refractored',
]

one_hot_columns = one_hot_columns_to_be_labeled#  + bucketize_columns (already inputed into the dnn as onehot by default...)


exclude_normalize_continuous_columns = [
    # Specific columns to exclude from normalization...
    # TODO: Should we exclude timestamp_month, day, and year??
] + one_hot_columns + label_columns + integerized_embedding_columns


normalize_continuous_columns = [c for c in all_feature_columns if c not in exclude_normalize_continuous_columns]
all_continuous_columns = [c for c in all_feature_columns if c not in one_hot_columns_to_be_labeled + label_columns + embedding_columns]

In [450]:
final_train_df = final_train_df[all_feature_columns + [LABEL]]

In [451]:
final_train_df.head()

,sport_count_5000,railroad_station_avto_min,mosque_count_500,cafe_count_5000_price_2500,trc_sqm_3000,big_road1_1line,build_count_brick,office_count_1500,healthcare_centers_raion,housing_fund_sqm,...,cafe_count_3000_price_500,perinatal_mort_per_1000_cap,load_of_teachers_preschool_per_teacher,cpi,church_count_5000,product_type,rent_price_3room_bus,cafe_count_5000_price_500,fin_res_per_cap,price_doc
0,52,6.905893,0,9,1419204,no,0.0,3,1,218.0,...,21,5.53,793.319561,354.0,22,Investment,77.93,39,226.214157,5850000
1,66,4.679745,0,15,491565,no,67.0,3,1,218.0,...,11,5.53,793.319561,354.0,29,Investment,77.93,49,226.214157,6000000
2,67,1.701420,0,10,52550,no,206.0,0,1,218.0,...,9,5.53,793.319561,354.0,27,Investment,77.93,29,226.214157,5700000
3,26,5.271136,0,11,205756,no,124.0,2,1,218.0,...,5,5.53,793.319561,353.2,4,Investment,94.02,7,226.214157,13100000
4,195,2.156284,0,319,2296870,no,643.0,93,4,218.0,...,266,5.53,793.319561,353.2,236,Investment,94.02,566,226.214157,16331452


In [452]:
final_train_df = final_train_df.convert_objects(convert_numeric=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [453]:
final_train_df.fillna(0, inplace=True)

## Create feature columns

In [454]:
SPARE_CAPACITY = 100

In [482]:
# First, continuous columns
tf_continuous_feature_columns = {}
for con_col in all_continuous_columns:
    if con_col in normalize_continuous_columns:
        mean = final_train_df[con_col].mean()
        std = final_train_df[con_col].std()
        fcol = tf.contrib.layers.real_valued_column(con_col, normalizer=lambda x : (x - mean) / std)
    else:
        fcol = tf.contrib.layers.real_valued_column(con_col)
    tf_continuous_feature_columns[con_col]  = fcol

In [483]:
tf_continuous_feature_columns

{'0_13_all': _RealValuedColumn(column_name='0_13_all', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e4384ba28>),
 '0_13_female': _RealValuedColumn(column_name='0_13_female', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e426fe938>),
 '0_13_male': _RealValuedColumn(column_name='0_13_male', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e1f523758>),
 '0_17_all': _RealValuedColumn(column_name='0_17_all', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e1f526de8>),
 '0_17_female': _RealValuedColumn(column_name='0_17_female', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e4382ad70>),
 '0_17_male': _RealValuedColumn(column_name='0_17_male', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e1f526aa0>),
 '0_6_all': _RealValuedColumn(column_name=

In [484]:
# Now, integerized columns
tf_integerized_feature_columns = {}
for integerized_col in integerized_embedding_columns:
    fcol = tf.contrib.layers.sparse_column_with_integerized_feature(integerized_col, bucket_size=max(final_train_df[integerized_col].values.ravel().tolist()) + SPARE_CAPACITY)
    tf_integerized_feature_columns[integerized_col] = fcol

In [485]:
tf_integerized_feature_columns

{'ID_big_road1': _SparseColumn(column_name='ID_big_road1', is_integerized=True, bucket_size=148, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_big_road2': _SparseColumn(column_name='ID_big_road2', is_integerized=True, bucket_size=158, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_bus_terminal': _SparseColumn(column_name='ID_bus_terminal', is_integerized=True, bucket_size=114, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_metro': _SparseColumn(column_name='ID_metro', is_integerized=True, bucket_size=323, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_railroad_station_avto': _SparseColumn(column_name='ID_railroad_station_avto', is_integerized=True, bucket_size=238, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_railroad_station_walk': _SparseColumn(column_name='ID_railroad_station_walk', is_integerized=True, bucket_size=233, lookup_config=None, combiner='sum', dtype=tf.int64),
 'ID_railroad_terminal': _SparseColumn(column_name='ID

In [486]:
# Now, embedding columns
tf_embedding_feature_columns = {}
for embed_col in embedding_columns:
    if embed_col in unlabeled_embedding_columns:
        bucket_size = len(final_train_df[embed_col].unique().ravel().tolist()) + SPARE_CAPACITY
        dimensions = determine_dimensions(len(final_train_df[embed_col].unique().ravel().tolist()) + int(SPARE_CAPACITY))
        fcol = tf.contrib.layers.scattered_embedding_column( embed_col, size=bucket_size * dimensions, dimension=dimensions, hash_key=tf.contrib.layers.SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY)
        tf_embedding_feature_columns[embed_col] = fcol
    else:
        # this is an integerized sparse column
        dimensions = determine_dimensions(len(final_train_df[embed_col].unique().ravel().tolist()) + int(SPARE_CAPACITY))
        fcol = tf.contrib.layers.embedding_column(tf_integerized_feature_columns[embed_col], dimension = dimensions)
        tf_embedding_feature_columns[embed_col] = fcol

In [487]:
tf_embedding_feature_columns

{'ID_big_road1': _EmbeddingColumn(sparse_id_column=_SparseColumn(column_name='ID_big_road1', is_integerized=True, bucket_size=148, lookup_config=None, combiner='sum', dtype=tf.int64), dimension=7, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f5e1f534fd0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, shared_embedding_name=None, shared_vocab_size=None, max_norm=None),
 'ID_big_road2': _EmbeddingColumn(sparse_id_column=_SparseColumn(column_name='ID_big_road2', is_integerized=True, bucket_size=158, lookup_config=None, combiner='sum', dtype=tf.int64), dimension=7, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f5e1f534f90>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, shared_embedding_name=None, shared_vocab_size=None, max_norm=None),
 'ID_bus_terminal': _EmbeddingColumn(sparse_id_column=_SparseColumn(column_name='ID_bus_terminal', is_integerized=True, bucket_size=114, lookup_config=None, combi

In [488]:
# Now, label columns
tf_label_feature_columns = {}
for label_col in label_columns + one_hot_columns_to_be_labeled:
    if label_col not in unlabeled_embedding_columns:
        # We've already covered embedding columns
        if label_col in integerized_label_columns:
            fcol = tf.contrib.layers.sparse_column_with_integerized_feature(label_col, bucket_size=max(final_train_df[label_col].values.ravel().tolist()) + SPARE_CAPACITY)
        else:
            num_unique = len(final_train_df[label_col].unique().ravel().tolist())
            fcol = tf.contrib.layers.sparse_column_with_hash_bucket(label_col, num_unique + SPARE_CAPACITY)
        tf_label_feature_columns[label_col] = fcol

In [489]:
tf_label_feature_columns

{'big_market_raion': _SparseColumn(column_name='big_market_raion', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string),
 'big_road1_1line': _SparseColumn(column_name='big_road1_1line', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string),
 'culture_objects_top_25': _SparseColumn(column_name='culture_objects_top_25', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string),
 'detention_facility_raion': _SparseColumn(column_name='detention_facility_raion', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string),
 'incineration_raion': _SparseColumn(column_name='incineration_raion', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string),
 'material': _SparseColumn(column_name='material', is_integerized=True, bucket_size=106, lookup_config=None, combiner='sum', dtype=tf.int64),
 'nuclear_reactor_raion': _S

In [490]:
# Now, bucketized columns
tf_bucket_feature_columns = {}
for bcol in bucketize_columns:
    boundaries = get_bin_boundaries(final_train_df, bcol)
    tf_bucket_feature_columns[bcol] = tf.contrib.layers.bucketized_column(tf.contrib.layers.real_valued_column(bcol + "_bucketized_feature"), boundaries)

In [491]:
tf_bucket_feature_columns

{'additional_education_km': _BucketizedColumn(source_column=_RealValuedColumn(column_name='additional_education_km_bucketized_feature', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), boundaries=(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0)),
 'apartment_fund_sqm': _BucketizedColumn(source_column=_RealValuedColumn(column_name='apartment_fund_sqm_bucketized_feature', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), boundaries=(226047.0, 228814.0, 231581.0, 234348.0)),
 'basketball_km': _BucketizedColumn(source_column=_RealValuedColumn(column_name='basketball_km_bucketized_feature', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), boundaries=(0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0)),
 'big_church_count_1000': _BucketizedColumn(source_column=_RealValuedColumn(column_name='big_church_count_1000_bucketized_feature

In [492]:
# Now, one-hot columns
tf_onehot_feature_columns = {}
for onehot_col in one_hot_columns:
    if onehot_col in one_hot_columns_to_be_labeled:
        fcol = tf.contrib.layers.one_hot_column(tf_label_feature_columns[onehot_col])
        tf_onehot_feature_columns[onehot_col] = fcol

In [493]:
tf_onehot_feature_columns

{'culture_objects_top_25': _OneHotColumn(sparse_id_column=_SparseColumn(column_name='culture_objects_top_25', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string)),
 'material': _OneHotColumn(sparse_id_column=_SparseColumn(column_name='material', is_integerized=True, bucket_size=106, lookup_config=None, combiner='sum', dtype=tf.int64)),
 'product_type': _OneHotColumn(sparse_id_column=_SparseColumn(column_name='product_type', is_integerized=False, bucket_size=102, lookup_config=None, combiner='sum', dtype=tf.string)),
 'state_refractored': _OneHotColumn(sparse_id_column=_SparseColumn(column_name='state_refractored', is_integerized=True, bucket_size=105, lookup_config=None, combiner='sum', dtype=tf.int64))}

In [494]:
# Some integerized feature columns have float ids that represent integers,
# so cast these float ids to integer ids
for c in integerized_embedding_columns + integerized_label_columns:
    final_train_df[c] =  final_train_df[c].astype(int)

In [495]:
TF_INPUT_FEATURE_COLS = tf_continuous_feature_columns.values() + tf_embedding_feature_columns.values() + tf_onehot_feature_columns.values() + tf_bucket_feature_columns.values()

In [496]:
TF_INPUT_FEATURE_COLS

[_RealValuedColumn(column_name='sport_count_5000', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e439b08c0>),
 _RealValuedColumn(column_name='radiation_km', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e4385bd70>),
 _RealValuedColumn(column_name='mosque_count_500', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e439b0c80>),
 _RealValuedColumn(column_name='cafe_count_5000_price_2500', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e439b0cf8>),
 _RealValuedColumn(column_name='trc_sqm_3000', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e4040d488>),
 _RealValuedColumn(column_name='build_count_brick', dimension=1, default_value=None, dtype=tf.float32, normalizer=<function <lambda> at 0x7f5e43564578>),
 _RealValuedColumn(column_name='office_count_1500', dimension=1, default_value=

In [497]:
TF_CONTINUOUS_COLS = all_continuous_columns + [c + "_bucketized_feature" for c in bucketize_columns]
TF_SPARSE_COLS = [c for c in all_feature_columns if c not in TF_CONTINUOUS_COLS]

In [498]:
TF_CONTINUOUS_COLS

['sport_count_5000',
 'railroad_station_avto_min',
 'mosque_count_500',
 'cafe_count_5000_price_2500',
 'trc_sqm_3000',
 'build_count_brick',
 'office_count_1500',
 'healthcare_centers_raion',
 'housing_fund_sqm',
 'heating_share',
 'catering_km',
 'office_count_500',
 'big_market_km',
 'museum_km',
 'gdp_quart_growth',
 'metro_km_avto',
 'radiation_km',
 'cafe_count_1000_price_1000',
 'build_count_foam',
 'cafe_count_3000_price_1000',
 'ice_rink_km',
 'balance_trade_growth',
 'leisure_count_1000',
 'cafe_avg_price_1000',
 'cafe_count_1500_price_1500',
 'mosque_count_2000',
 'trc_count_500',
 'industrial_km',
 'profitable_enterpr_share',
 'office_sqm_1000',
 '7_14_male',
 'load_of_teachers_school_per_teacher',
 'cafe_avg_price_500',
 'cafe_sum_1500_max_price_avg',
 'leisure_count_2000',
 'sport_objects_raion',
 'oil_chemistry_km',
 'museum_visitis_per_100_cap',
 'trc_count_2000',
 'students_state_oneshift',
 'mortgage_value',
 'cafe_count_500_price_1500',
 'cafe_count_3000_price_1500',

## Prepare training and testing datasets

In [499]:
def prepare_bucketized_column_in_dataset(df):
    global bucketize_columns
    for col in bucketize_columns:
        df[col + "_bucketized_feature"] = df[col]
    return df

In [500]:
final_train_df = prepare_bucketized_column_in_dataset(final_train_df)

In [501]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

In [502]:
train_split_df, val_split_df, test_split_df = train_validate_test_split(final_train_df)

## Data input function

In [519]:
def input_fn(df):
    global TF_CONTINUOUS_COLS, TF_SPARSE_COLS
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.       
    
    continuous_cols = {k: tf.reshape(tf.constant(df[k].values, dtype=tf.float64), [-1, 1])
                     for k in TF_CONTINUOUS_COLS}
    
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1])
                      for k in TF_SPARSE_COLS}
    
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols.items() + categorical_cols.items())
        
    # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL].values)
    
    # Returns the feature columns and the label.
    return feature_cols, label

## Model

In [520]:
def model_fn(features, targets, mode, params):
    global TF_INPUT_FEATURE_COLS
        
    # By default, let's do a 1024 -> 512 -> 256 -> 1 (with a dropout of 0.4 in between them)
    hidden_units = params.get('hidden_units', [1024, 512, 256])
    dropout = params.get('dropout', 0.4)
    last_dropout = params.get('last_dropout', 0.2)
    hidden_activation = params.get('hidden_activation', tf.nn.relu)
    lr = params.get("lr", 1e-4)
    opt = params.get("optimizer", "Adam")
    
    # Logic to do the following:
    
    # 1. Configure the model via TensorFlow operations
    
    input_layer = tf.contrib.layers.input_from_feature_columns(
        columns_to_tensors=features, feature_columns=TF_INPUT_FEATURE_COLS)
    
    for i, hidden_unit in enumerate(hidden_units):
        if i == 0:
            net = tf.contrib.layers.fully_connected(inputs=input_layer,
                                                 num_outputs=hidden_unit,
                                                 activation_fn=hidden_activation)
            if mode == tf.contrib.learn.ModeKeys.TRAIN:
                # Add droppout only during training
                if i == len(hidden_units) - 1:
                    net = tf.contrib.layers.dropout(net, keep_prob = 1 - last_dropout)
                else:
                    net = tf.contrib.layers.dropout(net, keep_prob = 1 - dropout)
        else:
            net = tf.contrib.layers.fully_connected(inputs=net,
                                                 num_outputs=hidden_unit,
                                                 activation_fn=hidden_activation)
            if mode == tf.contrib.learn.ModeKeys.TRAIN:
                # Add droppout only during training
                if i == len(hidden_units) - 1:
                    net = tf.contrib.layers.dropout(net, keep_prob = 1 - last_dropout)
                else:
                    net = tf.contrib.layers.dropout(net, keep_prob = 1 - dropout)
        
    # output layer
    output = tf.contrib.layers.fully_connected(inputs=net,
                                                 num_outputs=1,
                                                 activation_fn=tf.nn.relu)
    
    # Generate predictions
    predictions = tf.reshape(output, [-1])
    predictions = tf.cast(predictions, tf.float64)
    predictions_dict = {"prices": predictions}
    
    # 2. Define the loss function for training/evaluation
    """
    def root_mean_squared_logarithmic_error(y_true, y_pred):
    y_pred_log = K.log(y_pred + 1.)
    y_true_log = K.log(y_true + 1.)
    return K.sqrt(K.mean(K.square(y_pred_log - y_true_log), axis = -1))
    """
    targets = tf.cast(targets, tf.float64)
    
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.log1p(predictions) - tf.log1p(targets)), axis=-1))
    
    # 3. Define the training operation/optimizer
    train_op = tf.contrib.layers.optimize_loss(loss=loss,
                                               global_step=tf.contrib.framework.get_global_step(),
                                               learning_rate=lr,
                                               optimizer=opt)
    
    # Calculate root mean squared error as additional eval metric
    eval_metric_ops = {
      "rmse":
          tf.metrics.root_mean_squared_error(
              tf.cast(targets, tf.float64), predictions)
    }
        
    
    # 5. Return predictions/loss/train_op/eval_metric_ops in ModelFnOps object
    return tf.contrib.learn.ModelFnOps(
          mode=mode,
          predictions=predictions_dict,
          loss=loss,
          train_op=train_op,
          eval_metric_ops=eval_metric_ops)

In [530]:
# Set model params
model_params = {"lr":1e-3} # Use default

model_dir = '/mnt/h/Kaggle/Competitions/Russian Bank/models/' + version_str + '/tf_model'

# Instantiate Estimator
real_estate_model = tf.contrib.learn.Estimator(
    model_dir=model_dir,
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=30),
    model_fn=model_fn, params=model_params)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 30, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5e3de17210>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [531]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn = lambda : input_fn(val_split_df),
    eval_steps=1,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=20,
    every_n_steps=50)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


### Train the model

In [532]:
epochs = 1100

In [ ]:
real_estate_model.fit(input_fn = lambda : input_fn(train_split_df),
                     steps=epochs, monitors=[validation_monitor]
                     )

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 307 into /mnt/h/Kaggle/Competitions/Russian Bank/models/v2.1/tf_model/model.ckpt.
INFO:tensorflow:loss = 0.735164307396, step = 307
INFO:tensorflow:Starting evaluation at 2017-05-18-18:01:05
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-05-18-18:01:23
INFO:tensorflow:Saving dict for global step 307: global_step = 307, loss = 0.726333, rmse = 6.04299e+06
INFO:tensorflow:Validation (step 307): loss = 0.726333, global_step = 307, rmse = 6.04299e+06
INFO:tensorflow:Saving checkpoints for 308 into /mnt/h/Kaggle/Competitions/Russian Bank/models/v2.1/tf_model/model.ckpt.
INFO:tensorflow:Saving checkpoints for 309 into /mnt/h/Kaggle/Competitions/Russian Bank/models/v2.1/tf_model/model.ckpt.
INFO:tensorflow:Saving checkpoints for 310 into /mnt/h/Kaggle/Competitions/Russian Bank/models/v2.1/tf_model/model.ckpt.
INFO:tensorflow:Saving checkpoints for 311 into /mnt/h/Kaggle/Competition

## Generate Kaggle Submission file

In [326]:
kaggle_test = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/test.csv/test.csv")

In [327]:
kaggle_test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [328]:
macro_df = pd.read_csv("/mnt/h/Kaggle/Competitions/Russian Bank/data/macro.csv/macro.csv")

In [329]:
final_kaggle_df = pd.merge(kaggle_test, macro_df, on='timestamp')

In [330]:
final_kaggle_df = prepare_dataset(final_kaggle_df)

In [331]:
final_kaggle_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,timestamp_converted,timestamp_day,timestamp_month,timestamp_year,state_refractored
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,3
1,30475,2015-07-01,79.2,0.0,8,17,1,0.0,3,1.0,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,2
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,3
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,463938.0,0.0,0.0,0.0,234576.9,2015-07-01,1,7,2015,1


In [332]:
final_kaggle_df = final_kaggle_df[all_feature_columns ]

In [231]:
[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]

['sport_count_5000_bucketized_normalized_feature',
 'mosque_count_500_bucketized_normalized_feature',
 'cafe_count_5000_price_2500_bucketized_normalized_feature',
 'build_count_brick_bucketized_normalized_feature',
 'office_count_1500_bucketized_normalized_feature',
 'office_count_500_bucketized_normalized_feature',
 'cafe_count_1000_price_1000_bucketized_normalized_feature',
 'build_count_foam_bucketized_normalized_feature',
 'cafe_count_3000_price_1000_bucketized_normalized_feature',
 'leisure_count_1000_bucketized_normalized_feature',
 'cafe_count_1500_price_1500_bucketized_normalized_feature',
 'mosque_count_2000_bucketized_normalized_feature',
 'trc_count_500_bucketized_normalized_feature',
 'leisure_count_2000_bucketized_normalized_feature',
 'trc_count_2000_bucketized_normalized_feature',
 'cafe_count_500_price_1500_bucketized_normalized_feature',
 'cafe_count_3000_price_1500_bucketized_normalized_feature',
 'build_count_after_1995_bucketized_normalized_feature',
 'cafe_count_50

In [333]:
final_kaggle_df['old_education_build_share'] = final_kaggle_df.apply(lambda row: int(str(row['old_education_build_share']).replace(",","")), axis=1)

In [334]:
final_kaggle_df['modern_education_share'] = final_kaggle_df.apply(lambda row: int(str(row['modern_education_share']).replace(",","")), axis=1)

In [335]:
default_product_type = "Investment"

In [336]:
final_kaggle_df['product_type'] = final_kaggle_df.apply(lambda row: row['product_type'] if row['product_type'] != 0 else default_product_type, axis=1)

In [337]:
final_kaggle_df = transform_dataset(final_kaggle_df)

In [198]:
final_kaggle_df[[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]]

,sport_count_5000_bucketized_normalized_feature,mosque_count_500_bucketized_normalized_feature,cafe_count_5000_price_2500_bucketized_normalized_feature,build_count_brick_bucketized_normalized_feature,office_count_1500_bucketized_normalized_feature,office_count_500_bucketized_normalized_feature,cafe_count_1000_price_1000_bucketized_normalized_feature,build_count_foam_bucketized_normalized_feature,cafe_count_3000_price_1000_bucketized_normalized_feature,leisure_count_1000_bucketized_normalized_feature,...,nuclear_reactor_raion_normalized_feature,railroad_terminal_raion_normalized_feature,incineration_raion_normalized_feature,big_market_raion_normalized_feature,thermal_power_plant_raion_normalized_feature,big_road1_1line_normalized_feature,water_1line_normalized_feature,railroad_1line_normalized_feature,ecology_normalized_feature,sub_area_normalized_feature
0,1,0,0,2,0,0,0,1,1,0,...,1,1,1,1,1,1,1,1,5,38
1,1,0,1,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,3,103
2,2,0,1,2,1,0,1,0,1,1,...,1,1,1,1,2,1,1,1,4,84
3,1,0,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,3,105
4,1,0,1,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,3,103
5,3,0,2,2,1,1,1,0,1,0,...,1,1,1,1,1,1,1,2,5,24
6,1,0,1,1,1,0,1,0,1,1,...,1,1,1,1,1,1,1,1,2,124
7,1,0,1,0,0,0,0,0,1,0,...,1,1,1,2,1,1,1,1,3,102
8,1,0,1,2,0,0,0,0,1,0,...,1,1,1,1,1,1,1,1,3,136
9,1,0,1,0,0,0,0,0,1,0,...,1,1,1,1,1,1,1,1,3,103


In [199]:
[c for c in final_kaggle_df.columns.tolist() if "normalized" in c]

['sport_count_5000_bucketized_normalized_feature',
 'mosque_count_500_bucketized_normalized_feature',
 'cafe_count_5000_price_2500_bucketized_normalized_feature',
 'build_count_brick_bucketized_normalized_feature',
 'office_count_1500_bucketized_normalized_feature',
 'office_count_500_bucketized_normalized_feature',
 'cafe_count_1000_price_1000_bucketized_normalized_feature',
 'build_count_foam_bucketized_normalized_feature',
 'cafe_count_3000_price_1000_bucketized_normalized_feature',
 'leisure_count_1000_bucketized_normalized_feature',
 'cafe_count_1500_price_1500_bucketized_normalized_feature',
 'mosque_count_2000_bucketized_normalized_feature',
 'trc_count_500_bucketized_normalized_feature',
 'leisure_count_2000_bucketized_normalized_feature',
 'trc_count_2000_bucketized_normalized_feature',
 'cafe_count_500_price_1500_bucketized_normalized_feature',
 'cafe_count_3000_price_1500_bucketized_normalized_feature',
 'build_count_after_1995_bucketized_normalized_feature',
 'cafe_count_50

In [338]:
extract_normalized_feature(final_kaggle_df, "sub_area")

array([ 38, 103,  84, ..., 134,  79,  19])

In [339]:
kaggle_split_df_continuous_features = extract_normalized_features(final_kaggle_df)
kaggle_split_df_categorical_features = extract_categorical_features(final_kaggle_df, include_embedding_cols=False)
kaggle_split_df_embedding_features = extract_embedding_features(final_kaggle_df)
    
kaggle_split_df_embedding_features_final = {}
for k, v in kaggle_split_df_embedding_features.items():
    kaggle_split_df_embedding_features_final[k + "_input"] = v
       
kaggle_split_df_combined_inputs = combine_inputs([kaggle_split_df_continuous_features] 
                                                     +  kaggle_split_df_categorical_features.values())

In [340]:
kaggle_split_df_categorical_features

{'big_market_raion': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [2],
        [1]]), 'big_road1_1line': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]]), 'culture_objects_top_25': array([[ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        ..., 
        [ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  0.]]), 'detention_facility_raion': array([[1],
        [1],
        [1],
        ..., 
        [2],
        [1],
        [1]]), 'incineration_raion': array([[1],
        [1],
        [1],
        ..., 
        [1],
        [1],
        [1]]), 'material': array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.]]), 'nuclear_reactor_raion': array([[1],
        [1],
        [1],
        ..., 
   

In [341]:
kaggle_split_df_embedding_features

{'ID_big_road1': array([[ 2],
        [13],
        [10],
        ..., 
        [ 4],
        [23],
        [ 2]]), 'ID_big_road2': array([[38],
        [27],
        [ 1],
        ..., 
        [34],
        [52],
        [40]]), 'ID_bus_terminal': array([[ 9],
        [ 8],
        [ 3],
        ..., 
        [13],
        [ 6],
        [ 2]]), 'ID_metro': array([[ 45],
        [ 21],
        [ 44],
        ..., 
        [120],
        [ 53],
        [142]]), 'ID_railroad_station_avto': array([[39],
        [24],
        [59],
        ..., 
        [32],
        [27],
        [21]]), 'ID_railroad_station_walk': array([[ 39.],
        [ 24.],
        [ 68.],
        ..., 
        [  5.],
        [ 27.],
        [ 82.]]), 'ID_railroad_terminal': array([[32],
        [50],
        [ 5],
        ..., 
        [32],
        [32],
        [32]]), 'ecology': array([[5],
        [3],
        [4],
        ..., 
        [1],
        [4],
        [4]]), 'sub_area': array([[ 38],
        [103],


In [342]:
kaggle_split_df_continuous_features

array([[-0.838086  ,  0.03928507, -0.07009939, ...,  1.89720164,
        -0.48593907, -1.83239346],
       [-0.8812901 ,  0.16129706, -0.07009939, ...,  1.89720164,
        -0.49387983, -1.83239346],
       [ 0.39323092, -0.14841033, -0.07009939, ...,  1.89720164,
        -0.10478255, -1.83239346],
       ..., 
       [ 3.24470169, -0.19664223, -0.07009939, ...,  0.24523479,
         4.54850328, -1.83239346],
       [-0.21162651,  0.01626024, -0.07009939, ...,  0.24523479,
        -0.36682766, -1.83239346],
       [-0.23322857,  0.2272611 , -0.07009939, ...,  0.24523479,
        -0.20007168, -1.83239346]])

In [343]:
vs = []
for column in one_hot_columns:
    vals = final_kaggle_df[column].values.tolist()
    vals = [c if c in known_values[column] else unknown_label for c in vals]
    if unknown_label in vals:
        vs.append((column, vals))

In [344]:
[v[0] for v in vs]

[]

In [345]:
kaggle_preds = real_estate_price_model.predict(x=dict({
                    'main_input' : kaggle_split_df_combined_inputs
                }.items() + kaggle_split_df_embedding_features_final.items()))

In [346]:
kaggle_preds

array([[  3.07068355e+10],
       [  3.07081196e+10],
       [  3.07076506e+10],
       ..., 
       [  9.80176000e+06],
       [  1.45478860e+07],
       [  1.71274320e+07]], dtype=float32)

In [191]:
final_kaggle_df['price_doc'] = kaggle_preds

In [192]:
final_kaggle_df['price_doc']

0       4.093042e+10
1       4.093284e+10
2       4.093333e+10
3       4.093235e+10
4       4.092968e+10
5       4.093546e+10
6       4.093052e+10
7       4.092911e+10
8       4.093080e+10
9       4.093024e+10
10      4.093242e+10
11      4.092887e+10
12      4.092848e+10
13      4.092789e+10
14      4.093016e+10
15      4.093108e+10
16      4.093927e+10
17      4.093849e+10
18      4.093026e+10
19      4.093700e+10
20      4.093147e+10
21      4.093658e+10
22      4.093368e+10
23      4.093388e+10
24      4.093059e+10
25      4.093547e+10
26      4.093610e+10
27      4.093445e+10
28      4.092772e+10
29      4.093438e+10
            ...     
7632    1.344982e+07
7633    8.391098e+06
7634    1.654414e+07
7635    1.180624e+07
7636    6.630701e+06
7637    9.822717e+06
7638    7.509546e+06
7639    1.351762e+07
7640    1.002495e+07
7641    7.301856e+06
7642    1.362228e+07
7643    1.310399e+07
7644    1.443113e+07
7645    7.393820e+06
7646    1.267712e+07
7647    1.206970e+07
7648    1.271

In [193]:
4.093042e10

40930420000.0

In [271]:
final_kaggle_df[['id', 'price_doc']]

,id,price_doc
0,30474,5880385.00
1,30475,8747043.00
2,30476,6298690.00
3,30477,6858484.00
4,30478,5549552.00
5,30479,8727317.00
6,30480,4832307.00
7,30481,4594226.00
8,30482,5736204.00
9,30483,5148752.00


In [272]:
final_kaggle_df[['id', 'price_doc']].to_csv("/mnt/h/Kaggle/Competitions/Russian Bank/models/v" + str(version) + "/submission.csv", index=False) # output submission csv file